# CIFAR-10 Classification: Cats and Dogs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

## Load and Prepare Data

In [ ]:
# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, 
                                  download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, 
                                 download=True, transform=transform)

# Extract cats (class 3) and dogs (class 5)
def extract_cats_dogs(dataset):
    images = []
    labels = []
    for i, (img, label) in enumerate(dataset):
        if label in [3, 5]:  # 3=cat, 5=dog
            images.append(img)
            labels.append(1 if label == 5 else 0)  # 0=cat, 1=dog
    return torch.stack(images), torch.tensor(labels, dtype=torch.float32).unsqueeze(1)

x_train, y_train = extract_cats_dogs(train_dataset)
x_test, y_test = extract_cats_dogs(test_dataset)

print(f'Training set: {x_train.shape}, {y_train.shape}')
print(f'Test set: {x_test.shape}, {y_test.shape}')

## Visualize Data

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].imshow(x_train[0].permute(1, 2, 0))
axes[0].set_title('A cat')
axes[0].axis('off')

# Find a dog
dog_idx = (y_train == 1).nonzero()[0].item()
axes[1].imshow(x_train[dog_idx].permute(1, 2, 0))
axes[1].set_title('A dog')
axes[1].axis('off')
plt.tight_layout()
plt.show()

## Define Models

In [ ]:
class SimpleCNN(nn.Module):
    """Simple CNN for cat/dog classification"""
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout1(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)
print(model)

## Train Model

In [ ]:
batch_size = 128
epochs = 10

loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

def train_epoch(model, train_loader, loss_fn, optimizer, device):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        predicted = (outputs > 0.5).float().squeeze()
        total += labels.size(0)
        correct += (predicted == labels.squeeze()).sum().item()
    
    return train_loss / len(train_loader), 100 * correct / total

def evaluate(model, test_loader, loss_fn, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            test_loss += loss.item()
            predicted = (outputs > 0.5).float().squeeze()
            total += labels.size(0)
            correct += (predicted == labels.squeeze()).sum().item()
    
    return test_loss / len(test_loader), 100 * correct / total

train_losses = []
train_accs = []
val_losses = []
val_accs = []

for epoch in range(epochs):
    train_loss, train_acc = train_epoch(model, train_loader, loss_fn, optimizer, device)
    val_loss, val_acc = evaluate(model, test_loader, loss_fn, device)
    
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    print(f'Epoch {epoch+1}/{epochs}')
    print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

## Analyze Results

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(train_losses, label='Training Loss')
axes[0].plot(val_losses, label='Validation Loss')
axes[0].set_title('Model Loss')
axes[0].set_ylabel('Loss')
axes[0].set_xlabel('Epoch')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(train_accs, label='Training Accuracy')
axes[1].plot(val_accs, label='Validation Accuracy')
axes[1].set_title('Model Accuracy')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_xlabel('Epoch')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()